# PROJETO FINAL: TURMA 01 - GRUPO XX

# Integrantes:

i29089 - Nuno Pimentel

i31332 - Jesus Alban

i31433 - Matheus Anjos

i31476 - Diogo Gomes

# Objetivos:
Análise de tendencias relativamente à venda de jogos em diversas plataformas e em diversos países

In [276]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [277]:
#Variaveis Globais

## Ficheiro CSV de importação
csvFile = 'vgsales-12-4-2019.csv'

## Variaveis a adicionar ao ficheiro de exportação
excelFileName = 'ficheiro.xlsx'
tab1Name = 'Raw Data'
tab2Name = 'Platform Rank'
tab3Name = 'Percentage Rank'

# Configuração Pandas CSV
encodingName = 'utf8'
separator = ','

# Configuração do Pandas XLS
engineName = 'xlsxwriter'

# lista de colunas para a limpeza e pre-processamento dos dados
list_columns_delete = ['Rank','basename','VGChartz_Score','Critic_Score','User_Score','Total_Shipped','Last_Update','url','status','Vgchartzscore','img_url','ESRB_Rating']
list_columns_delete_lines_ifnull = ['Genre','Platform','Publisher','Developer','Global_Sales','Year']
list_columns_fill_nulls = ['NA_Sales', 'PAL_Sales', 'JP_Sales', 'Other_Sales']

Definição das funções relevantes para os objetivos propostos para o projeto final

In [278]:
# Função para remover a coluna de index de uma dataframe
def transform_remove_index(df):
    return df.drop(df.columns[0], axis=1)

In [279]:
# Função para eliminar colunas de um dataframe

def delete_columns(df, *columns_to_delete):
    columns_list = list(columns_to_delete)
    df = df.drop(columns=columns_list, errors='ignore')
    return df

In [280]:
# Função que filtra o dataframe com base nos valores não nulos de uma coluna

def filter_not_null(df,*columns_filter):
    try:
        filtered_df = df
        columns_list = list(columns_filter)
        for column in columns_list:
            if column in df.columns:
                filtered_df = filtered_df[filtered_df[column].notnull()]
        return filtered_df
    except:
        print(f'Column {column} not in DataFrame')

In [281]:
# Função que converte os valores de uma ou mais colunas em integer

def convert_int(df,*inColumn):
    columns_list = list(inColumn)
    for column in columns_list:
        if column in df.columns:
            df[column] = df[column].astype(int)
    return df

In [282]:
# Função que preenche os vazios de uma ou mais colunas

def fill_null(df, *column_name ,fill_value=0):
    for column in column_name:
        if column in df.columns:
            df[column] = df[column].fillna(fill_value)
    return df

In [283]:
# AVALIAR SE VAI PARA A VERSÃO FINAL

# Essa função é responsável por criar uma percentagem de linhas null
def count_null(df,inColumn):
    totalRows = len(df)
    countNull = 0
    for i, row in df.iterrows():
        if pd.isnull(row[inColumn]):
            countNull += 1
    percent = round((countNull * 100) / totalRows,2)
    return percent

In [284]:
# ADAPTAR O CÓDIGO PARA VÁRIOS DF E TABS

# Essa função recebe como parâmetro o pd.ExcelWriter, o file e a tab do excel e escreve no ficheiro
def write_xls(inWriter,df,inTab):
    # Falta código para escrever no ficheiro
    df.to_excel(inWriter, sheet_name=inTab,index=False)

In [285]:
# Função responsável somar valores de uma coluna e agrupar por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da soma
# No final é renomeada a coluna da soma para o nome que se quiser

def sum_column_by_group(df, column, *group_by_column, new_column_name='Sum'):
    group_by_list = list(group_by_column)
    
    df_sum_grouped = df.groupby(group_by_list)[column].sum().reset_index()
    df_sum_sorted = df_sum_grouped.sort_values(by=column, ascending = True).reset_index()
    df_sum_sorted.rename(columns={column: new_column_name}, inplace = True)
    return df_sum_grouped

In [286]:
# Função responsável por cacular a média dos valores de uma coluna e agrupar por uma ou mais colunas
# O reset index cria indexes para a nova lista criada
# A função sort_values ordena os dados por ordem descendente da média
# No final é renomeada a coluna da soma para o nome que se quiser

def mean_column_by_group(df, column, *group_by_column, new_column_name='Average'):
    group_by_list = list(group_by_column)

    df_mean_grouped = df.groupby(group_by_list)[column].mean().reset_index()
    df_mean_sorted = df_mean_grouped.sort_values(by=column, ascending = True).reset_index()
    df_mean_sorted.rename(columns={column: new_column_name}, inplace = True)
    return df_mean_sorted

In [287]:
# Função calcula a percentagem dos valores de uma coluna em função de um grupo

def percentage_by_group(df,inColumnMeasure, *group_by_column, new_column_name='percentage'):

    total_sum = df[inColumnMeasure].sum()
    Sum_per_group = sum_column_by_group(df, inColumnMeasure, *group_by_column)
    i = 0    
    while i < len(Sum_per_group):
        Sum_per_group.loc[i, new_column_name] = round(Sum_per_group.loc[i, inColumnMeasure] / total_sum * 100,2)
        i += 1

    Sum_per_group = Sum_per_group.sort_values(by=new_column_name, ascending = False).reset_index()
    Sum_per_group = Sum_per_group.head(5)

    return Sum_per_group


In [288]:
# Função com a configuração para o plot de um gráfico de linhas

def plot_trends(df, x_column, y_column, hue_column, title_name, xlabel_name, ylabel_name, legend_name, marker_type='o'):
    # Set the plot size
    plt.figure(figsize=(12, 6))
    
    # Create a line plot
    sns.lineplot(data=df, x=x_column, y=y_column, hue=hue_column, marker=marker_type)
    
    # Add titles and labels
    plt.title(title_name)
    plt.xlabel(xlabel_name)
    plt.ylabel(ylabel_name)
    plt.legend(title=legend_name)
    
    # Show the plot
    plt.show()

# Tarefa 1: Extração de dados

In [289]:
file = pd.read_csv(csvFile,sep=separator,encoding=encodingName)

# Tarefa 2:  Limpeza e Pré-Processamento de Dados

In [290]:
file = transform_remove_index(file)

file = delete_columns(file,*list_columns_delete)
file = filter_not_null(file,*list_columns_delete_lines_ifnull)
file = convert_int(file,'Year')
file = fill_null(file, *list_columns_fill_nulls)

# Tarefa 3: Exploratory Data Analysis (EDA)

In [291]:
#percentage_by_platform = percentage_by_group(file,'Global_Sales','Platform','Total_Global_Sales')

# Nuno
#sum_global_by_platform = sum_column_by_group(file, 'Global_Sales', 'Platform', 'Total_Global_Sales')
#mean_global_by_platform = mean_column_by_group(file, 'Global_Sales', 'Platform', 'Total_Global_Sales')

# Tarefa 4: Gravar dados em ficheiro Excel

In [292]:
writer = pd.ExcelWriter(excelFileName, engine=engineName)
write_xls(writer,file,tab1Name)
#write_xls(writer,sum_global_by_platform,tab2Name)
#write_xls(writer,percentage_by_platform,tab3Name)

writer.close()